# Optimization Algorithms

ML is iterative process, having good optimization algorithms allows for faster iterations thus faster results

better optimize convergence of model -> faster results

Overview:
- Mini-batches
- Exponentially weighted averages / Moving average + bias correction
- Adam Optimization
- learning rate decay

## Mini-batch

![img](https://miro.medium.com/1*I2BkYWA_OJzHWFo6kc0fmQ.jpeg)

vectorization allows for faster processing of batches

if m = 5,000,000 (examples) it will take a while to run a single forward and back pass

to speed up make mini batches ~1000 examples

X{500} = 500 mini-batches each containing 10,000 examples of original 5,000,000 example dataset


1 epoch = 1 pass through the whole data set, usual you want to take multiple passes through the dataset

![img](https://media.licdn.com/dms/image/v2/C4D12AQGhX6hIG-vdVw/article-inline_image-shrink_1500_2232/article-inline_image-shrink_1500_2232/0/1520236996533?e=1763596800&v=beta&t=1Vdp9pfcK78D0_wvC9gkRSFuOpF5w5-B6Bp6bFAh0Hc)

Mini batch is more noisy, sometimes loss goes up, but over time it will trend down. Some batches may be easier than others

mini-batch size is a hyperparameter
- Large mini-batch size -> regular batch;  low noise but takes alot of time for a single iteration
- mini-batch size -> stochastic gradient descent, looks at only 1 example at a time; very noisy but very fast

What works best is something inbetween, this value can be tunned like any other hyper parameter

Smaller training data sets can use whole batch 

typical minibatch sizes are 2^n (128,256,512)

Make sure it fits in CPU/GPU memory


## Exponentially weighted averages / Moving average

Compute average 

V[t] = B*V[t-1] + (1-B)Theta[t]

V[t] = average temperature over 1 / (B-1) Timestamps

Larger B(eta) :
- B = 0.98 averages over about 50 timestamps
- Produces smoother curve
- introduces latency as estimate changes gradually

Smaller B(eta) :
- B = 0.5 averages over about 2 timestamps

- more noise
- adapts faster to change

Provides a smooth trend estimate without storing long history; widely used in optimization algorithms for tracking moving averages of gradients or squared gradients.

```python

V = 0
data = []
weighted_averages = [V]

for x in data:
    V = beta * V + (1 - beta) * x
    weighted_averages.append(V)
    print(V)
```

Very memory and computationally cheap to compute

### Bias Correction

V[0] is initialized to 0, meaning that the first few datapoints will have a bias towards 0

additional operation of scaling up our weighter average for timestamp T using `V = V / ((1-beta)**t)`

this causes:
- for moving average with large value of beta -> V will be dramatically scaled up at small values of t
- for moving averages with small values of beta -> V will be less scaled up at small values of t

```python

V = 0
data = []
weighted_averages = [V]

for t,x in enumerate(data):
    V = (beta * V) + ((1 - beta) * x)
    V = V / ((1-beta)**t)
    weighted_averages.append(V)
    print(V)
```
```python

V = 0
data = []
weighted_averages = [V]

for t,x in enumerate(data):
    V_previous = beta * V
    V_current = (1-beta)* x
    V = V_previous + V_current
    V = V / ((1-beta)**t)
    weighted_averages.append(V)
    print(V)
```
Now when t is near 0 the previous averages will have near 0 affect, and as t increases it will have more affect on averages


# Gradient Descent  with momentum


Compute Exponentially weighted average of past gradients to calculate how much to tune each parameter by with goal of smoothing out any noise for faster convergence

algorithm maintains a moving average of the gradient vector and uses the weighted average to tune the parameters instead of the raw gradients


```python
Vdw,Vdb = 0
for i in iteration:
    dw,db = backprop()

    Vdw = beta * Vdw + (1-beta)* dw
    Vdb = beta * Vdb + (1-beta)* db

    w -= learning_rate * Vdw
    b -= learning_rate * Vdb
```

commonly used value for beta is 0.9: averages last 10 gradients

In practice bias correction isn't necessary as after 10 (beta=0.9) iterations the bias towards 0 would have disappeared

sometimes `Vdw = beta * Vdw + dw` is used (1-beta) omitted ; causes Vdw value to be scaled by (1-beta), causing beta to affect learning_rate if beta is changed

# RMSprop

Root Mean Square Propagation

keep track of a moving average of squared gradients

```python

Sdw,Sdb = 0
epsilon = 1e-8 # numerical stability for if Sdw is very small
for i in iteration:
    dw,db = backprop()

    Sdw = beta * Sdw + (1-beta)* (dw**2)
    Sdb = beta * Sdb + (1-beta)* (db**2)

    w -= learning_rate * (dw / np.sqrt(Sdw) + epsilon)
    b -= learning_rate * (db / np.sqrt(Sdb) + epsilon)
```

Parameters with consistently large gradients get smaller effective learning rates

Parameters with small or stable gradients get larger effective learning rates

This helps to reduce oscillations for steep directions while maintaining a fast learning rate for shallow directions

Allows for faster learning rates with less risk of divergence

# Adam Optimization Algo

takes momentum and RMSprop and combines


```python
Sdw,Sdb,Vdw,Vdb = 0

epsilon = 1e-8 # numerical stability for if gradient is very small, doesn't affect performance 

learning_rate = 0.001 # needs to be tuned

# these hyper params are often not tuned
beta_1 = 0.9 # Momentum beta
beta_2 = 0.999 # RMSProp beta

for i in iteration:
    dw,db = backprop()

    #RMSprop
    Sdw = beta_2 * Sdw + (1-beta_2)* (dw**2)
    Sdb = beta_2 * Sdb + (1-beta_2)* (db**2)

    #RMSprop bias correction
    Sdw_corrected = Sdw / (1-(beta_2**i))
    Sdb_corrected = Sdb / (1-(beta_2**i))

    #Momentum
    Vdw = beta_1 * Vdw + (1-beta_1)* dw
    Vdb = beta_1 * Vdb + (1-beta_1)* db

    #Momentum bias correction
    Vdw_corrected = Vdw / (1-(beta_1**i))
    Vdb_corrected = Vdb / (1-(beta_1**i))

    #here we combined RMSprop and Momentum
    w -= learning_rate * (Vdw_corrected / np.sqrt(Sdw_corrected) + epsilon)
    b -= learning_rate * (Vdb_corrected / np.sqrt(Sdb_corrected) + epsilon)
```



# learning rate decay

slowly reduce learning_rate during training

Assumption is that in initial iterations we can take larger steps, but in later iterations we should take smaller steps as we converge to the local minima


```python
# 2 hyper parameters
decay_rate = 
initial_learning_rate

learning_rate = 1 / (1+decay_rate * epoch_number) * initial_learning_rate
```
### alternative methods
exponetial decay: `learning_rate = 0.95**(epoch_number) * initial_learning_rate`

`learning_rate = (some_constant / np.sqrt(epoch_num) ) * init_learning_rate `

Manual decay: for some long running times, you can just manually lessen learning rate when you see fit

Fixed Interval Scheduling: at specific timestamps use specifically defined learning rates

scheduled lr delay: `learning_rate = (1/ (1 + decay_rate * np.floor(epoch_num / time_interval))) * learning_rate0`


# Local Optima

![img](https://www.allaboutlean.com/wp-content/uploads/2018/08/Local-Global-Optimum.png)

Thinking about local optimas on a low dimensional space would lead us to assume there are many local minimas 

Most critical points in high dimensional space are saddle points and not local minimas due to the chance that every dimension (2000 dimensional space) at a critical point is a minima is very low

thus unlikely to get stuck in a bad local optima for models with many parameters

Plateaus, a region where the gradient is near 0, takes a while to escape. this is why optimizers like adam are useful

